# LSTM System for Frequency Extraction from Mixed Signals

**M.Sc. Assignment: Developing an LSTM System for Frequency Extraction**

Dr. Segal Yoram | November 2025

---

## Table of Contents

1. [Introduction](#1.-Introduction)
2. [Dataset Generation](#2.-Dataset-Generation)
3. [Model Architecture](#3.-Model-Architecture)
4. [Training](#4.-Training)
5. [Evaluation](#5.-Evaluation)
6. [Results Visualization](#6.-Results-Visualization)
7. [Conclusions](#7.-Conclusions)

## 1. Introduction

### Problem Statement

Given a mixed signal **S(t)** composed of 4 sinusoidal frequencies with random noise:

$$S(t) = \frac{1}{4} \sum_{i=1}^{4} A_i(t) \cdot \sin(2\pi \cdot f_i \cdot t + \phi_i(t))$$

where:
- $A_i(t) \sim \text{Uniform}(0.8, 1.2)$ (varies at **each** sample)
- $\phi_i(t) \sim \text{Uniform}(0, 2\pi)$ (varies at **each** sample)
- $f_i \in \{1, 3, 5, 7\}$ Hz

**Goal**: Train an LSTM to extract pure frequency components:

$$\text{Target}_i(t) = \sin(2\pi \cdot f_i \cdot t)$$

### Key Implementation Requirements

- **Sequence Length**: L = 1 (critical pedagogical requirement)
- **State Reset**: Internal state (h_t, c_t) **must be reset** between batches
- **Training Dataset**: seed #1
- **Test Dataset**: seed #2 (completely different noise)

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import os
import json

# Import our custom modules
from src.data_generator import create_train_and_test_datasets, load_dataset
from src.model import FrequencyExtractorLSTM, create_dataloader
from src.trainer import train_model, load_trained_model, get_training_config
from src.evaluator import evaluate_model, evaluate_by_frequency, compute_metrics, print_metrics, save_metrics
from src.visualizer import create_all_visualizations, plot_training_curve

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 2. Dataset Generation

We generate two datasets:
1. **Training set**: Using random seed #1
2. **Test set**: Using random seed #2

Both datasets contain 40,000 samples (10,000 time points × 4 frequencies).

**Critical**: Amplitude and phase vary **at each sample** to create challenging noise conditions.

In [ ]:
# Create datasets (will skip if already exist)
if not (os.path.exists('data/train_dataset.npz') and os.path.exists('data/test_dataset.npz')):
    print("Generating datasets...\n")
    train_data, test_data = create_train_and_test_datasets(save_dir='data')
else:
    print("Datasets already exist. Loading...\n")
    train_data = load_dataset('data/train_dataset.npz')
    test_data = load_dataset('data/test_dataset.npz')

In [ ]:
# Visualize dataset structure
print("\nDataset Structure:")
print("="*70)
print(f"Training samples: {len(train_data['S'])}")
print(f"Test samples: {len(test_data['S'])}")
print(f"Frequencies: {train_data['frequencies']}")
print(f"Sampling rate: {train_data['sampling_rate']} Hz")
print(f"Time range: {train_data['time'][0]:.3f} - {train_data['time'][-1]:.3f} seconds")
print("="*70)

# Show sample data
print("\nSample rows (first 5):")
print(f"Time (s) | S[t] (noisy) | C (selection) | Target (pure)")
print("-"*70)
for i in range(5):
    print(f"{train_data['time'][i]:.3f}    | {train_data['S'][i]:8.4f}    | {train_data['C'][i]} | {train_data['targets'][i]:8.4f}")

In [ ]:
# Visualize mixed signal and components
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

time_plot = train_data['time'][:1000]

# 1. Mixed noisy signal
axes[0].plot(time_plot, train_data['S'][:1000], 'g-', alpha=0.7, linewidth=1)
axes[0].set_ylabel('Amplitude', fontsize=11)
axes[0].set_title('Mixed Noisy Signal S(t)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# 2. Pure target for f2 = 3 Hz
target_f2 = train_data['targets'][10000:11000]  # f2 samples
axes[1].plot(time_plot, target_f2, 'b-', linewidth=2)
axes[1].set_ylabel('Amplitude', fontsize=11)
axes[1].set_title('Pure Target: f2 = 3 Hz', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# 3. All 4 pure targets
for i, freq in enumerate([1, 3, 5, 7]):
    target = train_data['targets'][i*10000:(i*10000)+1000]
    axes[2].plot(time_plot, target, label=f'f{i+1} = {freq} Hz', alpha=0.7)
axes[2].set_xlabel('Time (seconds)', fontsize=11)
axes[2].set_ylabel('Amplitude', fontsize=11)
axes[2].set_title('All Pure Target Frequencies', fontsize=13, fontweight='bold')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/dataset_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print("Dataset visualization saved to results/plots/dataset_overview.png")

## 3. Model Architecture

### LSTM Network Structure

```
Input: [S[t], C1, C2, C3, C4]  (5 features)
   ↓
LSTM Layer (64 hidden units)
   ↓
Fully Connected Layer
   ↓
Output: Target_i[t]  (1 value)
```

### Critical State Management

For **L = 1** (sequence length = 1), the internal state **(h_t, c_t) is reset** for each batch:

```python
hidden = None  # Forces zero initialization
output, hidden = lstm(input, hidden)
```

This demonstrates that the LSTM can learn frequency patterns through internal memory management **alone**, without relying on sequential dependencies.

In [ ]:
# Get training configuration
config = get_training_config()
print("Training Configuration:")
print("="*70)
for key, value in config.items():
    print(f"{key:20s}: {value}")
print("="*70)

In [ ]:
# Create model
model = FrequencyExtractorLSTM(
    input_size=config['input_size'],
    hidden_size=config['hidden_size'],
    num_layers=config['num_layers'],
    dropout=config['dropout']
)

print("\nModel Architecture:")
print("="*70)
print(model)
print("="*70)
print(f"Total parameters: {model.get_num_parameters():,}")
print("="*70)

## 4. Training

Training the model with:
- **Optimizer**: Adam (lr=0.001)
- **Loss Function**: Mean Squared Error (MSE)
- **Batch Size**: 64
- **Early Stopping**: Patience = 10 epochs
- **State Management**: Reset for each batch (critical!)

In [ ]:
# Create dataloaders
train_loader = create_dataloader(
    train_data['S'],
    train_data['C'],
    train_data['targets'],
    batch_size=config['batch_size'],
    shuffle=True
)

test_loader = create_dataloader(
    test_data['S'],
    test_data['C'],
    test_data['targets'],
    batch_size=config['batch_size'],
    shuffle=False
)

print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Train model (or load if already trained)
model_path = 'models/best_model.pth'

if not os.path.exists(model_path):
    print("Starting training...\n")
    history = train_model(
        model,
        train_loader,
        test_loader,
        num_epochs=config['num_epochs'],
        learning_rate=config['learning_rate'],
        device=config['device'],
        save_path=model_path,
        patience=config['patience'],
        verbose=True
    )
else:
    print("Model already trained. Loading...\n")
    model, checkpoint = load_trained_model(model, model_path, device=config['device'])
    history = checkpoint.get('history', {})

In [ ]:
# Plot training curve
if history:
    plot_training_curve(history, 'results/plots/training_curve.png')
    
    # Display training progress
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    epochs = history['epochs']
    ax1.plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    ax1.plot(epochs, history['test_loss'], 'r-', label='Test Loss', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('MSE Loss', fontsize=12)
    ax1.set_title('Training Progress', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    if 'learning_rates' in history:
        ax2.plot(epochs, history['learning_rates'], 'g-', linewidth=2)
        ax2.set_xlabel('Epoch', fontsize=12)
        ax2.set_ylabel('Learning Rate', fontsize=12)
        ax2.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nFinal Training Loss: {history['train_loss'][-1]:.6f}")
    print(f"Final Test Loss: {history['test_loss'][-1]:.6f}")
else:
    print("No training history available.")

## 5. Evaluation

Evaluate the trained model on both training and test sets to:
1. Calculate MSE metrics
2. Check generalization (MSE_test ≈ MSE_train)
3. Generate predictions for visualization

In [ ]:
# Evaluate on training set
print("Evaluating on training set...")
train_mse, train_predictions, train_targets = evaluate_model(
    model, train_loader, device=config['device']
)

# Evaluate on test set
print("Evaluating on test set...")
test_mse, test_predictions, test_targets = evaluate_model(
    model, test_loader, device=config['device']
)

print(f"\nTraining MSE: {train_mse:.6f}")
print(f"Test MSE: {test_mse:.6f}")

In [ ]:
# Compute and display metrics
metrics = compute_metrics(train_mse, test_mse)
print_metrics(metrics)

# Save metrics
save_metrics(metrics, 'results/metrics.json')

In [ ]:
# Evaluate by frequency
print("\nEvaluating performance per frequency...\n")

test_results_by_freq = evaluate_by_frequency(
    model,
    test_data['S'],
    test_data['C'],
    test_data['targets'],
    batch_size=config['batch_size'],
    device=config['device']
)

print("Per-Frequency MSE (Test Set):")
print("="*70)
frequencies = [1, 3, 5, 7]
for freq_idx, freq in enumerate(frequencies):
    freq_mse = test_results_by_freq[freq_idx]['mse']
    print(f"f{freq_idx+1} = {freq} Hz: MSE = {freq_mse:.6f}")
print("="*70)

## 6. Results Visualization

Generate the required visualizations:
1. **Graph 1**: Single frequency detailed comparison (e.g., f2 = 3 Hz)
2. **Graph 2**: All four frequencies extraction

In [ ]:
# Create all visualizations
create_all_visualizations(
    history,
    metrics,
    test_results_by_freq,
    test_data['time'],
    frequencies=[1, 3, 5, 7],
    output_dir='results/plots'
)

In [ ]:
# Display Graph 1: Single Frequency Comparison (f2 = 3 Hz)
from IPython.display import Image
print("\nGraph 1: Single Frequency Detailed Comparison (f2 = 3 Hz)")
display(Image('results/plots/freq_comparison.png'))

In [ ]:
# Display Graph 2: All Frequencies
print("\nGraph 2: All Four Frequencies Extraction")
display(Image('results/plots/all_frequencies.png'))

In [ ]:
# Display Metrics Comparison
print("\nMetrics Comparison")
display(Image('results/plots/metrics_comparison.png'))

## 7. Conclusions

### Summary of Results

This project successfully demonstrates:

1. **LSTM as Frequency Filter**: The LSTM network learned to extract pure frequency components from heavily noised mixed signals.

2. **State Management**: With sequence length L=1 and proper state reset (hidden=None), the network learned frequency patterns through internal memory management alone.

3. **Noise Robustness**: Despite amplitude variations (±20%) and random phase shifts at **every sample**, the LSTM successfully recovered clean sinusoids.

4. **Generalization**: The model generalizes well to completely different noise (seed #2), as evidenced by MSE_test ≈ MSE_train.

### Key Learnings

- **Internal State**: The LSTM's hidden state (h_t, c_t) acts as an adaptive filter that learns to track frequency-periodic patterns.

- **Conditional Regression**: The one-hot vector C successfully conditions the network to extract different frequencies from the same input signal.

- **Training Stability**: Proper hyperparameters (learning rate, gradient clipping) ensure stable convergence.

### Assignment Requirements Checklist

✅ Created 2 datasets (train seed #1, test seed #2)  
✅ Noise varies at EACH sample (A_i(t), φ_i(t))  
✅ Built LSTM with proper architecture  
✅ Reset internal state between samples (L=1)  
✅ Trained model to low MSE  
✅ Evaluated generalization (MSE_test ≈ MSE_train)  
✅ Generated required visualizations  
✅ Demonstrated frequency extraction success  

### Future Work

Potential extensions:
1. Experiment with L > 1 (sliding window approach)
2. Compare with other architectures (GRU, Transformer)
3. Test with different noise models
4. Real-time streaming implementation
5. Multi-frequency extraction (extract multiple frequencies simultaneously)

---

**Assignment Complete!** 🎉